<a href="https://colab.research.google.com/github/Harsh-017/ECGGuard/blob/main/Arrhythmia_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle kagglehub[pandas-datasets]

^C
Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/82.7 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.7 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.7 kB ? eta -:--:--
     -------------- ----------------------- 30.7/82.7 kB 186.2 kB/s eta 0:00:01
     ---------------------------- --------- 61.4/82.7 kB 326.1 kB/s eta 0:00:01
     -------------------------------------- 82.7/82.7 kB 385.4 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Secure the file

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json

import os
import json

# Move the file to the correct location
os.makedirs('/root/.kaggle', exist_ok=True)
with open("kaggle.json", "r") as f:
    kaggle_creds = json.load(f)

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_creds, f)

os.chmod("/root/.kaggle/kaggle.json", 600)

In [ ]:
import subprocess

# List available datasets (optional)
subprocess.run(["kaggle", "datasets", "list"])

# Download ECG dataset
subprocess.run(["kaggle", "datasets", "download", "-d", "erhmrai/ecg-image-data", "--unzip", "-p", "/kaggle/input"])

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!kaggle datasets download -d erhmrai/ecg-image-data -p /content --unzip

In [ ]:
import shutil
extracted_folder = "/kaggle/input/ecg-image-data"
final_dataset_path = "/kaggle/input/ecg_data"
if os.path.exists(extracted_folder):
    shutil.move(extracted_folder, final_dataset_path)

In [ ]:
import os
import zipfile
import kaggle

# Set up Kaggle dataset information
DATASET = "erhmrai/ecg-image-data"
KAGGLE_INPUT_PATH = "/kaggle/input"
KAGGLE_WORKING_PATH = "/kaggle/working"
DATASET_ZIP_PATH = os.path.join(KAGGLE_INPUT_PATH, "ecg-image-data.zip")

# Ensure input and working directories exist
os.makedirs(KAGGLE_INPUT_PATH, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, exist_ok=True)

# Download the dataset using Kaggle API
print(f"Downloading dataset: {DATASET}...")
os.system(f"kaggle datasets download -d {DATASET} -p {KAGGLE_INPUT_PATH} --unzip")

# Extract if zip file exists
if os.path.exists(DATASET_ZIP_PATH):
    print("Extracting dataset...")
    with zipfile.ZipFile(DATASET_ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(KAGGLE_INPUT_PATH)
    os.remove(DATASET_ZIP_PATH)

print("Dataset downloaded and extracted successfully!")


In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# ✅ Fix: Ensure TensorFlow 2.x is used
print("TensorFlow Version:", tf.__version__)

# Define dataset paths
DATASET_PATH = "/content/ECG_Image_data"  # Change this to your dataset directory
train_path = os.path.join(DATASET_PATH, "train")
test_path = os.path.join(DATASET_PATH, "test")

# Preprocessing and Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize images
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load Images
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)

# ✅ Load Pretrained VGG19 Model
base_model = VGG19(input_shape=(224, 224, 3), weights="imagenet", include_top=False)

# Freeze VGG19 Layers
for layer in base_model.layers:
    layer.trainable = False

# Add Custom Layers
x = Flatten()(base_model.output)
x = Dense(128, activation="relu")(x)
x = Dense(1, activation="sigmoid")(x)  # Binary Classification

# Create Model
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Train Model
history = model.fit(train_generator, validation_data=test_generator, epochs=5)

# Save Model
model.save("ecg_vgg19_model.h5")

print("✅ Model training complete! Saved as ecg_vgg19_model.h5")


The system cannot find the path specified.


OSError: [WinError 1314] A required privilege is not held by the client: '/kaggle/input' -> '..\\input'

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '../input/ecg-image-data/ECG_Image_data/train'
valid_path = '../input/ecg-image-data/ECG_Image_data/test'

In [ ]:
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable = False

In [ ]:
folders = glob('../input/ecg-image-data/ECG_Image_data/train/*')
folders ////// check this

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('../input/ecg-image-data/ECG_Image_data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('../input/ecg-image-data/ECG_Image_data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=3,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
plt.plot(r.history['accuracy'], label='train accuracy')
plt.plot(r.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_accuracy')

In [ ]:
import tensorflow as tf
from keras.models import load_model


In [ ]:
model.save('model_vgg19.h5')

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
for i in range(len(y_pred)):
  if y_pred[i]==1:
    print(i)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_vgg19.h5')

In [ ]:
img=image.load_img('../input/ecg-image-data/ECG_Image_data/test/F/F380.png',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

In [ ]:
x.shape

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
if(a==1):
    print("Uninfected")
else:
    print("Infected")